In [1]:
import numpy as np
import json
from nltk.tokenize import word_tokenize
%cd "E:\NLP\QA"

E:\NLP\QA


In [2]:
with open("documents.json") as f:
    doc = json.load(f)
with open("training.json") as f:
    train = json.load(f)

In [3]:
tokens = []
for d in doc:
    paragraphs = d["text"]
    for paragraph in paragraphs:
        tokens += word_tokenize(paragraph.lower())
print(len(set(tokens)))
for sample in train:
    question = sample["question"]
    tokens += word_tokenize(question.lower())
print(len(set(tokens)))
tokens = set(tokens)

98289
100757


In [6]:
embedding = []
mapping = dict()
with open("glove.6B.50d.txt","r",encoding="utf-8", errors="ignore") as f:
    for line in f:
        data = line.split()
        word = data[0]
        if word in tokens:
            embedding.append(np.array([float(i) for i in data[1:]]))
            mapping[word] = len(mapping)
embedding.append(np.random.uniform(-0.5,0.5,50))
mapping["<UNK>"] = len(mapping)
embedding.append(np.zeros(50))
mapping["<PAD>"] = len(mapping)

In [7]:
np.save("embedding", embedding)

In [8]:
len(mapping)

72499

In [9]:
embedding = np.load("embedding.npy").astype("float32")

In [63]:
with open("mapping.json","w") as f:
    json.dump(mapping, f)

In [13]:
embedding[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)